In [1]:
import pandas as pd
import re

In [4]:
def contains_any_substring_v8(title, include_list, exclude_list, never_allowed, combination_allowed, combination_denied):
    title = title.lower()
    if any(substring in title for substring in combination_allowed) and any(substring in title for substring in combination_denied):
        for substring in combination_denied:
            title = title.replace(substring, '')
    if any(substring in title for substring in ('歌ってみた', '歌みた')) and '雑談' in title:
        title = title.replace('歌ってみた', '').replace('歌みた', '')
    if any(never in title for never in never_allowed):# or all(item not in title for item in include_list)
        return False
    replaced_title = re.sub('|'.join(exclude_list), '', title)
    parsed_title = '|'.join([substring for substring in re.split('\W+', replaced_title) if (not substring.endswith('sing') or substring == 'sing')])    
    include_regex = '|'.join(include_list)
    return any(re.search(include_regex, substring) for substring in re.split('\|', parsed_title))

In [13]:
def dataframe_columns_to_lists(df):
    """
    Converts each column in a DataFrame into a list.

    Parameters:
    - df (pd.DataFrame): The input DataFrame.

    Returns:
    - dict: A dictionary containing column names as keys and lists as values.
    """
    column_lists = {}
    for column in df.columns:
        column_lists[column] = df[column].dropna().tolist()
    return column_lists

In [248]:
def process_schedule_data(path, filters, min_scheduled_time='2023-05-01', 
                          max_scheduled_time = '2024-01-01'):
    # Read the CSV file
    df = pd.read_csv(path)
    
    if 'isVideo' in df.columns:
        df = df[df['isVideo']==False]
        #df.drop('isVideo', axis=1, inplace=True)

    # Apply filtering based on the 'Title' column
    df['is_valid'] = df['Title'].apply(
        lambda title: contains_any_substring_v8(
            title,
            include_list=filters['include_list'],
            exclude_list=filters['exclude_list'],
            never_allowed=filters['never_allowed'],
            combination_allowed=filters['combination_allowed'],
            combination_denied=filters['combination_denied']
        )
    )

    # Filter out invalid rows
    df = df[df['is_valid']]
    print(f"Dataframe length:{len(df)}")

    # Convert 'Scheduled Time' to datetime and filter by the specified date
    df['Scheduled Time'] = pd.to_datetime(df['Scheduled Time'])
    
    min_scheduled_time = pd.to_datetime(min_scheduled_time)
    max_scheduled_time = pd.to_datetime(max_scheduled_time)
    df = df[df['Scheduled Time'].between(min_scheduled_time, 
                                                  max_scheduled_time, inclusive='both')]
    
    # Group by 'Channel Name'
    grouped_by_channel = df.groupby('Channel Name')

    # Create a list to store the results
    result_data = []

    # Loop through groups and append results to the list
    for k, g in grouped_by_channel:
        result_data.append({'Channel Name': k, 'Count': len(g), 'Min Scheduled Time': g['Scheduled Time'].min()})

    # Create a DataFrame from the list of results
    result_df = pd.DataFrame(result_data)

    return df, result_df

In [249]:
def process_and_merge_data(paths, filters):
    counts = []
    raws = []
    # Assuming process_schedule_data is a function that reads data from a path and applies filters
    for path in paths:
        raw_df, count_df = process_schedule_data(path, filters)
        raws.append(raw_df)
        counts.append(count_df)
    
    result_df = pd.concat(counts, axis=0)
    # Group by 'Channel Name' and aggregate the 'Count' sum and 'Min Scheduled Time' minimum
    merged = result_df.groupby('Channel Name').agg(
        {'Count': 'sum', 'Min Scheduled Time': 'min'}
    ).reset_index()
    
    # Sort the merged DataFrame by 'Count' in descending order
    merged = merged.sort_values('Count', ascending=False)
    
    return raws, merged

In [250]:
def plot_count_histogram(data):
    import plotly.graph_objects as go
    # Create a histogram using plotly
    fig = go.Figure()

    fig.add_trace(go.Histogram(x=data['Count'], marker_color='skyblue'))

    # Set layout options
    fig.update_layout(
        title='Histogram of Counts',
        xaxis=dict(title='Count'),
        yaxis=dict(title='Frequency'),
        template='simple_white'  # Use the 'simple_white' template
    )

    # Show the plot
    fig.show()


In [251]:
def calculate_count_statistics(data):
    # Calculate statistics for the 'Count' column
    count_stats = data['Count'].describe()

    # Calculate IQR separately
    iqr = count_stats['75%'] - count_stats['25%']

    # Return the results as a dictionary
    results = {
        "Minimum": count_stats['min'],
        "Maximum": count_stats['max'],
        "Mean": count_stats['mean'],
        "Standard Deviation": count_stats['std'],
        "IQR (Interquartile Range)": iqr
    }

    return results

In [252]:
def create_filters_from_dataframe(df):
    filters = dataframe_columns_to_lists(df)
    
    # Add additional filters
    filters['combination_allowed'] = ['振り返', 'ありがとう', '後日談', 'アフタートーク',
                                      'release', '感想', '打ち上げ', '裏話', 'ふりかえり',
                                      'behind', '明日', '描', 'Project DIVA MEGA39', 'talk',
                                      '疲れ様', '回顧', '公開直前', '前夜祭',]

    filters['combination_denied'] = ['歌リレー', '歌枠リレー', 'live', 'ライブ', 'song', 
                                     'MV', '歌ってみた', '歌みた']
    
    return filters

In [253]:
parent_dir = '/kaggle/input/liveuta-2023-05to2023-12'
filter_path = f'{parent_dir}/title_filter - title_filter.csv'
path1 = f'{parent_dir}/vuta_schedule_warehouse_20230522 - 1.csv'
path2 = f'{parent_dir}/vuta_schedule_warehouse_08 - 1.csv'
path3 = f'{parent_dir}/vuta_schedule_warehouse - vuta_schedule_warehouse.csv'

In [254]:
filters = create_filters_from_dataframe(pd.read_csv(filter_path))
raws, result = process_and_merge_data([path1, path2, path3], filters)

Dataframe length:9237
Dataframe length:18002
Dataframe length:9241


In [255]:
temp_list = []
for df in raws:
    temp_list.append(df[['Channel Name', 'Scheduled Time']])
    
temp_df = pd.concat(temp_list, axis=0)
temp_df['Scheduled Time'] = pd.to_datetime(temp_df['Scheduled Time'])

In [256]:
temp_df['date_only'] = temp_df['Scheduled Time'].apply(lambda x: x.date())

In [257]:
temp_df

,Channel Name,Scheduled Time,date_only
0,란츄 코토부키,2023-05-22 16:00:00,2023-05-22
1,미레아 셸츠,2023-05-22 16:30:00,2023-05-22
2,오니가시마 쿠로메,2023-05-22 17:00:00,2023-05-22
3,카나미 유유,2023-05-22 17:00:00,2023-05-22
4,오리히메 하루카,2023-05-22 17:40:00,2023-05-22
...,...,...,...
8795,아스유메 카나에,2023-12-31 23:40:00,2023-12-31
8796,Monologue / 사쿠라&리리아&마오,2023-12-31 23:40:23,2023-12-31
8799,히야미즈 누루메,2023-12-31 23:55:08,2023-12-31
8800,코하타로,2023-12-31 23:56:11,2023-12-31


In [259]:
temp_df['date_only'].value_counts()

date_only
2023-12-07    415
2023-12-09    354
2023-12-06    342
2023-12-10    339
2023-12-25    330
             ... 
2023-08-07     94
2023-05-22     65
2023-10-02     55
2023-10-01     38
2024-01-01      1
Name: count, Length: 225, dtype: int64

In [244]:
plot_count_histogram(result)

In [165]:
calculate_count_statistics(result)

{'Minimum': 1.0,
 'Maximum': 214.0,
 'Mean': 21.604551045510455,
 'Standard Deviation': 26.713623448285286,
 'IQR (Interquartile Range)': 24.0}

In [ ]:
21.6 - 1.5*24

In [264]:
result['dates'] = result['Min Scheduled Time'].apply(
    lambda x: (pd.to_datetime('2024-01-01')-x).days)

In [265]:
result['CPD'] = result['Count']/result['dates']

In [270]:
a = result[result['Count']>22].sort_values('Count', ascending=False).head(50).reset_index(drop=True)

In [278]:
b = result[result['Count']>22].sort_values('CPD', ascending=False).head(50).reset_index(drop=True)

In [279]:
a

,Channel Name,Count,Min Scheduled Time,dates,CPD
0,오가 루키,217,2023-05-22 23:00:00,223,0.973094
1,시라타마 우타노,201,2023-05-23 19:00:00,222,0.905405
2,하루카 미코토,165,2023-05-23 20:00:00,222,0.743243
3,노와르/NoiR,164,2023-05-25 23:00:00,220,0.745455
4,츠키시로 세실,157,2023-05-23 20:00:00,222,0.707207
5,MUS1CA 무지카,151,2023-05-23 23:00:00,222,0.680180
6,니쥬나,147,2023-06-03 21:30:00,211,0.696682
7,뭉못슈,147,2023-05-22 17:45:00,223,0.659193
8,라티오 유이리스,146,2023-05-22 19:00:00,223,0.654709
9,히메노 노에,141,2023-05-23 20:00:00,222,0.635135


In [280]:
b

,Channel Name,Count,Min Scheduled Time,dates,CPD
0,오가 루키,217,2023-05-22 23:00:00,223,0.973094
1,키츠네,95,2023-09-22 22:33:58,100,0.950000
2,시라타마 우타노,201,2023-05-23 19:00:00,222,0.905405
3,토키니와 란제,41,2023-11-08 19:00:00,53,0.773585
4,노와르/NoiR,164,2023-05-25 23:00:00,220,0.745455
5,하루카 미코토,165,2023-05-23 20:00:00,222,0.743243
6,츠키시로 세실,157,2023-05-23 20:00:00,222,0.707207
7,니쥬나,147,2023-06-03 21:30:00,211,0.696682
8,MUS1CA 무지카,151,2023-05-23 23:00:00,222,0.680180
9,아오이,87,2023-08-22 23:00:43,131,0.664122


In [281]:
aa = a['Channel Name'].tolist()

In [282]:
bb = b['Channel Name'].tolist()

In [283]:
common_elements = list(set(aa) & set(bb))

print(common_elements)

['아마하네 미란', '로쿠리 샤오', '키츠네', '타카나시 유토하', '노와르/NoiR', '츠키시로 세실', '우타노 네루', '아오이 스이', '야야무기', '모에미 단', '아와유키 유리', '시라타마 우타노', '샤이루', '오가 루키', 'MUS1CA 무지카', '코하쿠 루나', '뭉못슈', '모치모치 사쿠라', '유즈하 마쿠라', '디노스 티라노', '오니가시마 쿠로메', '이누즈키 렌', '히메노 노에', '모캉', '아몬 유우쿠', '시라카와 시라세', '미타니 미쿠', '엔마 루리', 'Monologue / 사쿠라&리리아&마오', '우미츠키 셸', '모나카 모코코', '하네무라 키미카', '라티오 유이리스', '마요', '사오토메 아즈키', '쿠온 나츠메', '요시카', '니쥬나', '하루카 미코토']
